In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torchvision.models import resnet50, ResNet50_Weights
from torch.utils.data import DataLoader, TensorDataset
from PIL import Image
import pandas as pd


In [7]:
image_dir = "Images/"
train_txt = "train_test_files/5_folders_cross_validations_files/cross_validation_1/train_1.txt"
test_txt = "train_test_files/5_folders_cross_validations_files/cross_validation_1/test_1.txt"


In [8]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),  
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # https://stackoverflow.com/questions/58151507/why-pytorch-officially-use-mean-0-485-0-456-0-406-and-std-0-229-0-224-0-2
])

In [9]:
def load_data(txt_file, image_dir):
    data = pd.read_csv(txt_file, delim_whitespace=True, header=None)
    image_paths = [image_dir + img_name for img_name in data[0]]
    scores = data[1].values
    return image_paths, scores

def create_dataloader(image_paths, scores, batch_size=64):
    images = []
    for img_path in image_paths:
        image = Image.open(img_path).convert('RGB')
        image = transform(image)
        images.append(image)

    images = torch.stack(images)
    scores = torch.tensor(scores, dtype=torch.float32)

    dataset = TensorDataset(images, scores)


    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return dataloader

In [10]:
train_image_paths, train_scores = load_data(train_txt, image_dir)
test_image_paths, test_scores = load_data(test_txt, image_dir)

train_loader = create_dataloader(train_image_paths, train_scores)
test_loader = create_dataloader(test_image_paths, test_scores)

In [18]:
model = resnet50(weights=ResNet50_Weights.DEFAULT)

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)

In [12]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

In [21]:
num_epochs = 10  
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    progress = 0
    for inputs, targets in train_loader:
        progress += 1
        print(f'Progress: {progress}/{len(train_loader)}', end='\r')
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')
    

model.eval()
running_loss = 0.0

with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), targets)
        running_loss += loss.item() * inputs.size(0)

test_loss = running_loss / len(test_loader.dataset)
print(f'Test Loss: {test_loss:.4f}')



Epoch 1/10, Loss: 9.4581
Epoch 2/10, Loss: 9.4531
Epoch 3/10, Loss: 9.4571
Epoch 4/10, Loss: 9.4586
Epoch 5/10, Loss: 9.4572
Epoch 6/10, Loss: 9.4579


KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), 'resnet50_finetuned.pth')
